In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Arrhythmia"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Arrhythmia/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Arrhythmia/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Arrhythmia/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Arrhythmia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Check if there's a suitable cohort directory for Arrhythmia
print(f"Looking for a relevant cohort directory for {trait}...")

# Check available cohorts
available_dirs = os.listdir(tcga_root_dir)
print(f"Available cohorts: {available_dirs}")

# Arrhythmia is a cardiac condition, so we should look for heart/cardiac-related cohorts
cardiac_related_terms = ['heart', 'cardiac', 'cardiovascular', 'thoracic', 'chest']

# First check for direct heart/cardiac related cohorts
cardiac_related_dirs = [d for d in available_dirs if any(term in d.lower() for term in cardiac_related_terms)]
print(f"Cardiac-related cohorts: {cardiac_related_dirs}")

# If no direct heart-related cohorts, we might need to look at:
# 1. General datasets that might include cardiac data
# 2. Datasets that affect organs near the heart
# 3. Datasets where cardiac function might be measured as part of standard evaluation
if not cardiac_related_dirs:
    print("No direct cardiac cohorts found. Looking for possible related cohorts...")
    # Lung, thoracic, or chest area studies might include cardiac data
    possible_related_cohorts = [d for d in available_dirs 
                               if any(term in d.lower() for term in ['lung', 'thoracic', 'chest', 'thymoma'])]
    print(f"Possible related cohorts: {possible_related_cohorts}")
    
    if possible_related_cohorts:
        # Lung studies often include cardiac measures
        selected_cohort = [d for d in possible_related_cohorts if 'lung' in d.lower()][0] if any('lung' in d.lower() for d in possible_related_cohorts) else possible_related_cohorts[0]
    else:
        print(f"No suitable cohort found for {trait}.")
        # Mark the task as completed by recording the unavailability
        validate_and_save_cohort_info(
            is_final=False,
            cohort="TCGA",
            info_path=json_path,
            is_gene_available=False,
            is_trait_available=False
        )
        # Exit the script early since no suitable cohort was found
        selected_cohort = None
else:
    selected_cohort = cardiac_related_dirs[0]

if selected_cohort:
    print(f"Selected cohort: {selected_cohort}")
    
    # Get the full path to the selected cohort directory
    cohort_dir = os.path.join(tcga_root_dir, selected_cohort)
    
    # Get the clinical and genetic data file paths
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic data file: {os.path.basename(genetic_file_path)}")
    
    # Load the clinical and genetic data
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Print the column names of the clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Basic info about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")


Looking for a relevant cohort directory for Arrhythmia...
Available cohorts: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA


Clinical data columns:
['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_0_no_1_yes', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_LUAD', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_LUAD', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# 2. Load the clinical data file
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(
    os.path.join(tcga_root_dir, 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)'))
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')

# Extract and preview the candidate columns for age
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Extract and preview the candidate columns for gender
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Candidate age columns:")
print(candidate_age_cols)
print("\nAge data preview:")
print(age_preview)

print("\nCandidate gender columns:")
print(candidate_gender_cols)
print("\nGender data preview:")
print(gender_preview)


Candidate age columns:
['age_at_initial_pathologic_diagnosis', 'days_to_birth']

Age data preview:
{'age_at_initial_pathologic_diagnosis': [67.0, 67.0, 72.0, 72.0, 77.0], 'days_to_birth': [-24477.0, -24477.0, -26615.0, -26615.0, -28171.0]}

Candidate gender columns:
['gender']

Gender data preview:
{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Select appropriate columns for age and gender
age_col = None
gender_col = None

# Evaluate age columns
if 'age_at_initial_pathologic_diagnosis' in ['age_at_initial_pathologic_diagnosis', 'days_to_birth']:
    # Check if the column has meaningful values (not all None or NaN)
    preview_values = [67.0, 67.0, 72.0, 72.0, 77.0]
    if any(v is not None and not pd.isna(v) for v in preview_values):
        age_col = 'age_at_initial_pathologic_diagnosis'

# Evaluate gender columns
if 'gender' in ['gender']:
    # Check if the column has meaningful values (not all None or NaN)
    preview_values = ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']
    if any(v is not None and not pd.isna(v) for v in preview_values):
        gender_col = 'gender'

# Print the chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# 1. Extract and standardize clinical features
# Use tcga_select_clinical_features which will automatically create the trait variable and add age/gender if provided
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data if not already loaded
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

linked_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Print preview of clinical features
print("Clinical features (first 5 rows):")
print(linked_clinical_df.head())

# 2. Process gene expression data
print("\nProcessing gene expression data...")
# Load genetic data from the same cohort directory
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Check gene data shape
print(f"Original gene data shape: {genetic_df.shape}")

# Save a version of the gene data before normalization (as a backup)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
genetic_df.to_csv(out_gene_data_file.replace('.csv', '_original.csv'))

# We need to transpose genetic data so genes are rows and samples are columns for normalization
gene_df_for_norm = genetic_df.copy().T

# Try to normalize gene symbols - adding debug output to understand what's happening
print("Attempting to normalize gene symbols...")
try:
    normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm)
    print(f"Gene data shape after normalization: {normalized_gene_df.shape}")
    
    # Check if normalization returned empty DataFrame
    if normalized_gene_df.shape[0] == 0:
        print("WARNING: Gene symbol normalization returned an empty DataFrame.")
        print("Using original gene data instead of normalized data.")
        # Use original data instead - samples as rows, genes as columns
        normalized_gene_df = genetic_df
    else:
        # If normalization worked, transpose back to original orientation
        normalized_gene_df = normalized_gene_df.T
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    print("Using original gene data instead.")
    normalized_gene_df = genetic_df

# Save gene data
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
# TCGA data uses the same sample IDs in both datasets
print("\nLinking clinical and genetic data...")
print(f"Clinical data shape: {linked_clinical_df.shape}")
print(f"Genetic data shape: {normalized_gene_df.shape}")

# Find common samples between clinical and genetic data
common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.columns))
print(f"Number of common samples: {len(common_samples)}")

if len(common_samples) == 0:
    print("ERROR: No common samples found between clinical and genetic data.")
    # Use is_final=False mode which doesn't require df and is_biased
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True
    )
    print("The dataset was determined to be unusable for this trait due to no common samples. No data files were saved.")
else:
    # Filter clinical data to only include common samples
    linked_clinical_df = linked_clinical_df.loc[list(common_samples)]
    
    # Create linked data by merging
    linked_data = pd.concat([linked_clinical_df, normalized_gene_df[list(common_samples)].T], axis=1)
    
    print(f"\nLinked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first few columns):")
    display_cols = [trait, 'Age', 'Gender'] + list(linked_data.columns[3:5])
    print(linked_data[display_cols].head())
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"\nData shape after handling missing values: {linked_data.shape}")
    
    # 5. Check for bias in trait and demographic features
    print("\nChecking for bias in features:")
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Validate and save cohort info
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=len(linked_data.columns) > 3,  # More than just trait/age/gender columns
        is_trait_available=trait in linked_data.columns,
        is_biased=is_trait_biased,
        df=linked_data,
        note="Data from TCGA Lung Squamous Cell Carcinoma cohort used as proxy for Arrhythmia-related cardiac gene expression patterns."
    )
    
    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to: {out_data_file}")
        
        # Also save clinical data separately
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_columns = [col for col in linked_data.columns if col in [trait, 'Age', 'Gender']]
        linked_data[clinical_columns].to_csv(out_clinical_data_file)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    else:
        print("The dataset was determined to be unusable for this trait. No data files were saved.")

Clinical features (first 5 rows):
                 Arrhythmia   Age  Gender
sampleID                                 
TCGA-18-3406-01           1  67.0     1.0
TCGA-18-3406-11           0  67.0     1.0
TCGA-18-3407-01           1  72.0     1.0
TCGA-18-3407-11           0  72.0     1.0
TCGA-18-3408-01           1  77.0     0.0

Processing gene expression data...


Original gene data shape: (20530, 553)


Attempting to normalize gene symbols...
Gene data shape after normalization: (0, 20530)
Using original gene data instead of normalized data.


Gene data saved to: ../../output/preprocess/Arrhythmia/gene_data/TCGA.csv

Linking clinical and genetic data...
Clinical data shape: (626, 3)
Genetic data shape: (20530, 553)
Number of common samples: 553

Linked data shape: (553, 20533)
Linked data preview (first 5 rows, first few columns):
                 Arrhythmia   Age  Gender  ARHGEF10L     HIF3A
TCGA-56-A62T-01           1  78.0     1.0  -1.102992 -4.457126
TCGA-85-8351-01           1  72.0     1.0  -1.391792 -0.616526
TCGA-43-3394-01           1  52.0     1.0  -1.188092 -0.863726
TCGA-LA-A7SW-01           1  71.0     1.0  -2.085992 -2.273426
TCGA-56-7580-01           1  84.0     1.0  -1.926792  3.154774



Data shape after handling missing values: (553, 20533)

Checking for bias in features:
For the feature 'Arrhythmia', the least common label is '0' with 51 occurrences. This represents 9.22% of the dataset.
The distribution of the feature 'Arrhythmia' in this dataset is fine.

Quartiles for 'Age':
  25%: 62.0
  50% (Median): 68.0
  75%: 73.0
Min: 39.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 144 occurrences. This represents 26.04% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Performing final validation...


Linked data saved to: ../../output/preprocess/Arrhythmia/TCGA.csv
Clinical data saved to: ../../output/preprocess/Arrhythmia/clinical_data/TCGA.csv
